# 卷积层里的多输入多输出通道

## 实现一下多输入通道互相关运算

In [3]:
def corr2d(X, K):
    h, w = K.shape
    """比如说K 3x3 , X 16x16 , 那Y 16-3+1=14 """
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j: j+w] * K).sum() # 很清楚了，这就是卷积！
    return Y

In [17]:



import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    return sum(corr2d(x, k) for x, k in zip(X, K)) # 这里的for会对最外层做遍历





## 验证互相关运算的输出

In [12]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
X.shape, K.shape

(torch.Size([2, 3, 3]), torch.Size([2, 2, 2]))

In [11]:
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [18]:
list(corr2d(x, k) for x, k in zip(X, K))

[tensor([[19., 25.],
         [37., 43.]]),
 tensor([[37., 47.],
         [67., 77.]])]

In [20]:
for x, k in zip(X, K):
    print('1')

1
1


## 计算多个通道的输出的互相关函数

In [31]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
X.shape, K.shape

(torch.Size([2, 3, 3]), torch.Size([2, 2, 2]))

In [25]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [32]:
K

tensor([[[0., 1.],
         [2., 3.]],

        [[1., 2.],
         [3., 4.]]])

In [33]:
K+1

tensor([[[1., 2.],
         [3., 4.]],

        [[2., 3.],
         [4., 5.]]])

In [34]:
K = torch.stack((K,K+1,K+2),0)
K.shape

torch.Size([3, 2, 2, 2])

In [35]:
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## 1x1卷积

In [43]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    print(f'orig X:{X.shape}')
    X = X.reshape((c_i, h*w))
    print(f'reshape X:{X.shape}')
    print(' ')
    print(f'orig K:{K.shape}')
    K = K.reshape((c_o, c_i))
    print(f'reshape K:{K.shape}')
    print(' ')
    Y = torch.matmul(K, X)
    print(f'orig Y:{Y.shape}')
    print(f'reshape Y:{Y.reshape((c_o, h, w)).shape}')
    return Y.reshape((c_o, h, w))

In [38]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
X.shape, K.shape

(torch.Size([3, 3, 3]), torch.Size([2, 3, 1, 1]))

In [44]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

orig X:torch.Size([3, 3, 3])
reshape X:torch.Size([3, 9])
 
orig K:torch.Size([2, 3, 1, 1])
reshape K:torch.Size([2, 3])
 
orig Y:torch.Size([2, 9])
reshape Y:torch.Size([2, 3, 3])


In [45]:
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6